In [28]:
import numpy as np
import json

In [29]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertForNextSentencePrediction

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenized input
text = "Who was Jim Henson ? Jim Henson was a puppeteer"
tokenized_text = tokenizer.tokenize(text)

# Convert token to vocabulary indices
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
# Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
segments_ids = [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]

# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

model = BertForNextSentencePrediction.from_pretrained('bert-base-uncased')
model.eval()

# Predict all tokens
predictions = model(tokens_tensor, segments_tensors)

# confirm we were able to predict 'henson'
# predicted_index = torch.argmax(predictions[0, masked_index]).item()
# predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
# assert predicted_token == 'henson'


01/17/2019 23:55:08 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /Users/daniter/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
01/17/2019 23:55:08 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /Users/daniter/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
01/17/2019 23:55:08 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file /Users/daniter/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /var/folders/xx/8h5l1j

In [30]:
def test(sent1, sent2):
    # Tokenized input
    sent1_tokens =  tokenizer.tokenize(sent1)
    sent2_tokens =  tokenizer.tokenize(sent2)
    tokenized_text = tokenizer.tokenize(sent1 + " " + sent2)

    # Convert token to vocabulary indices
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    # Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
    segments_ids = [0]*len(sent1_tokens) + [1]*len(sent2_tokens)

    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    model = BertForNextSentencePrediction.from_pretrained('bert-base-uncased')
    model.eval()

    # Predict all tokens
    #predictions = model(torch.cat((tokens_tensor, tokens_tensor), 0), torch.cat((segments_tensors, segments_tensors), 0))
    predictions = model(tokens_tensor,  segments_tensors)
    #print (predictions)
    return predictions[0][0] - predictions[0][1]


In [33]:
sent1 = "The boy went to the store and bought a fish."
sent2 = "Toys are always more fun than homework"
print(test(sent1,sent2))

01/17/2019 23:55:51 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /Users/daniter/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
01/17/2019 23:55:51 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file /Users/daniter/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /var/folders/xx/8h5l1j614vv5wmbx9fbj69wm0000gn/T/tmpk0l90y0r
01/17/2019 23:55:54 - INFO - pytorch_pretrained_bert.modeling -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  

tensor(-4.5192, grad_fn=<SubBackward0>)


# Part 2: Can we rewrite questions with answer?

In [4]:
with open("dataset/dev-v2.0.json", 'r') as handle:
    jdata = json.load(handle)
    data = jdata['data']

In [6]:
contexts = []
questions = []
unanswerable = []
answerable = []
for i in range(len(data)):
    section = data[i]['paragraphs']
    for sec in section:
        context = sec['context']
        contexts.append(context)
        qas = sec['qas']
        for j in range(len(qas)):
            question = qas[j]['question']
            questions.append(question)
            label = qas[j]['is_impossible']
            if label:
                unanswerable.append((len(contexts)-1, len(questions)-1))
            else:
                answerable.append((len(contexts)-1, len(questions)-1))

In [27]:
i = np.random.choice(range(len(unanswerable)))
c_i, q_i = unanswerable[i]
print(contexts[c_i])
print("##"*10)
print(questions[q_i])
print(test(contexts[c_i], questions[q_i]))

NameError: name 'np' is not defined

In [28]:
%%time
i = np.random.choice(range(len(answerable)))
c_i, q_i = answerable[i]
print(contexts[c_i])
print("##"*10)
print(questions[q_i])
print(test(contexts[c_i], questions[q_i]))

After World War II, under a Communist regime set up by the conquering Soviets, the "Bricks for Warsaw" campaign was initiated, and large prefabricated housing projects were erected in Warsaw to address the housing shortage, along with other typical buildings of an Eastern Bloc city, such as the Palace of Culture and Science, a gift from the Soviet Union. The city resumed its role as the capital of Poland and the country's centre of political and economic life. Many of the historic streets, buildings, and churches were restored to their original form. In 1980, Warsaw's historic Old Town was inscribed onto UNESCO's World Heritage list.
####################
What were the structures built by the Soviets typical of?
tensor(5.2437, grad_fn=<SubBackward0>)
CPU times: user 5.91 s, sys: 972 ms, total: 6.88 s
Wall time: 6.46 s


In [10]:
## Get all the next sentence predictions for Squad
from torch.utils.data import DataLoader
from tqdm import tqdm, trange
import torch
from examples.run_qa_lm_finetune import BERTDataset
from pytorch_pretrained_bert import BertTokenizer, BertForNextSentencePrediction


In [11]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
max_seq_length = 128
train_dataset = BERTDataset("dataset/dev-v2.0.json", tokenizer, seq_len=max_seq_length,
                                     on_memory=True)

01/17/2019 23:58:34 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /Users/daniter/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
Loading Squad: 100%|██████████| 35/35 [00:00<00:00, 4549.00it/s]


In [12]:
train_batch_size = 32
train_dataloader = DataLoader(train_dataset, batch_size=train_batch_size)


In [13]:
model = BertForNextSentencePrediction.from_pretrained('bert-base-uncased')
model.eval()

def test_multi(sent1_list, sent2_list, model):
    # Predict all tokens
    predictions = model(sent1_list, sent2_list)
    #print (predictions)
    return predictions

01/17/2019 23:58:36 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /Users/daniter/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
01/17/2019 23:58:36 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file /Users/daniter/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /var/folders/xx/8h5l1j614vv5wmbx9fbj69wm0000gn/T/tmpgcz0song
01/17/2019 23:58:39 - INFO - pytorch_pretrained_bert.modeling -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")
for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration")):
    batch = tuple(t.to(device) for t in batch)
    input_ids, input_mask, segment_ids, lm_label_ids, is_next = batch
    print(test_multi(input_ids, segment_ids, model))
    break
    

Iteration:   0%|          | 0/186 [00:00<?, ?it/s]01/17/2019 23:58:41 - INFO - examples.run_qa_lm_finetune -   *** Example ***
01/17/2019 23:58:41 - INFO - examples.run_qa_lm_finetune -   guid: 0
01/17/2019 23:58:41 - INFO - examples.run_qa_lm_finetune -   tokens: [CLS] the norman ##s ( norman : no ##ur ##man ##ds ; french : norman ##ds ; latin : norman ##ni ) were the people who in the 10th and 11th centuries gave their name to normandy , a region in france . they were descended from norse ( " norman " comes from " norse ##man " ) raiders and pirates from denmark , iceland and norway who , under their leader roll ##o , agreed to swear fe ##al ##ty to king charles iii of west fran ##cia . through generations of assimilation and mixing with the native frankish and roman - gaul ##ish populations , their descendants would gradually merge with the carol ##ing ##ian - [SEP] [MASK] what country is normandy located ? [SEP]
01/17/2019 23:58:41 - INFO - examples.run_qa_lm_finetune -   input_ids

01/17/2019 23:58:42 - INFO - examples.run_qa_lm_finetune -   Is next sentence label: 0 
01/17/2019 23:58:42 - INFO - examples.run_qa_lm_finetune -   *** Example ***
01/17/2019 23:58:42 - INFO - examples.run_qa_lm_finetune -   guid: 3
01/17/2019 23:58:42 - INFO - examples.run_qa_lm_finetune -   tokens: [CLS] the norman ##s ( norman : no ##ur ##man ##ds ; french : norman ##ds ; latin : norman ##ni ) were the people who in the 10th and 11th centuries gave their name to normandy , a region in france . they were descended from norse ( " norman " comes from " norse ##man " ) raiders and pirates from denmark , iceland and norway who , under their leader roll ##o , agreed to swear fe ##al ##ty to king charles iii of west fran ##cia . through generations of assimilation and mixing with the native frankish and roman - gaul ##ish populations , their descendants would gradually merge with the carol ##ing ##ian - based [SEP] who was the norse leader ? [SEP]
01/17/2019 23:58:42 - INFO - examples.run

tensor([[ 5.9191, -5.6865],
        [ 6.1558, -5.9778],
        [ 3.1260, -2.0181],
        [ 6.2322, -6.1348],
        [ 6.2923, -6.1853],
        [ 3.4570, -2.5800],
        [ 6.2002, -6.0441],
        [ 6.1843, -6.1073],
        [ 6.3741, -6.2566],
        [ 6.3830, -6.2630],
        [ 6.3597, -6.2517],
        [ 6.4111, -6.3297],
        [ 5.7352, -5.1993],
        [ 5.5352, -4.9631],
        [ 6.0173, -5.7273],
        [ 6.2732, -6.1182],
        [ 5.5546, -5.0173],
        [ 6.2370, -6.0499],
        [ 6.2640, -6.1043],
        [ 6.4198, -6.3765],
        [ 6.3961, -6.3139],
        [ 5.4202, -4.8951],
        [ 6.3817, -6.3047],
        [ 6.4175, -6.3831],
        [ 6.4491, -6.3593],
        [ 3.4403, -2.2939],
        [ 4.5917, -3.9043],
        [ 3.7436, -2.5898],
        [ 6.2464, -6.0598],
        [ 5.9751, -5.7331],
        [ 5.3952, -4.6008],
        [ 6.3138, -6.2016]], grad_fn=<AddmmBackward>)


In [8]:
print(test_multi(input_ids, segment_ids, model))


tensor([[ 6.1265, -5.9967],
        [ 6.2441, -6.0882],
        [ 6.1823, -5.9816],
        [ 5.5371, -4.9621],
        [ 6.3424, -6.2344],
        [ 5.2208, -4.7500],
        [ 6.2002, -6.0441],
        [-2.1503,  4.6283],
        [ 6.2251, -6.0152],
        [ 6.3551, -6.2389],
        [ 6.2981, -6.1389],
        [ 6.3772, -6.2689],
        [-2.1128,  4.4824],
        [ 6.0253, -5.7061],
        [ 6.1853, -6.0180],
        [ 6.2732, -6.1182],
        [ 5.8994, -5.5320],
        [ 6.3360, -6.2196],
        [ 6.4162, -6.3704],
        [ 6.4157, -6.3625],
        [ 6.4249, -6.3512],
        [ 5.7378, -5.4084],
        [ 6.1788, -5.9174],
        [ 6.3058, -6.1782],
        [ 5.4878, -4.7047],
        [ 6.1572, -5.9165],
        [ 5.6450, -5.2820],
        [ 5.8424, -5.4555],
        [ 4.9847, -4.1712],
        [ 5.3419, -4.6825],
        [ 6.3137, -6.0859],
        [ 6.3559, -6.2961]], grad_fn=<AddmmBackward>)


In [9]:
old1, old2 = input_ids, segment_ids

In [15]:
assert old1 == input_ids

RuntimeError: bool value of Tensor with more than one value is ambiguous

In [29]:
for i in range(5,32):
    for j in range(128):
        assert input_ids[i,j] ==  old1[i,j], (input_ids[i,j], old1[i,j], i, j)

AssertionError: (tensor(1999), tensor(103), 5, 121)

In [21]:
old1.shape

torch.Size([32, 128])